In [16]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "GPU" #"PCI_BUS_ID"   #if like me you do not have a lot of memory in your GPU
os.environ["CUDA_VISIBLE_DEVICES"] = "GPU" #then these two lines force keras to use your CPU
#import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization, Activation, Softmax
from tensorflow.keras.utils import to_categorical
from tensorflow import keras 
import numpy as np
#np.random.seed(66)

In [17]:
import tensorflow as tf

In [18]:
class SppnetLayer(keras.layers.Layer):
    '''This layer takes an input tensor and pools the tensor
      in local spatial bins.
      This layer uses Max pooling.
      It accepts input in tensorflow format. # channels last

    # Input
        list of filter in form [x,y,z] 
    # Input shape : 5d tensor [None, X,Y,Z channels]
    # Output shape : 3d tensor [None,pooled dim, channels] 

    '''
    def __init__(self, filters = [1], **kwargs):
        self.filters = filters
        super(SppnetLayer, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        length = 0;
        for f_size in self.filters:
            length+= (f_size*f_size)
        return (input_shape[0],length*input_shape[4])
      
    def get_config(self):
        config = {'filters': self.filters}
        base_config = super(SppnetLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
      output = []
      for f_size in self.filters:
        win_size = K.int_shape(inputs)[1]/f_size
        print(win_size)
        win_size = int(win_size)
        for x_start in range(0,f_size):
          for y_start in range(0,f_size):
            for z_start in range(0,f_size):
                X = int(x_start*win_size)
                Y = int(y_start*win_size)
                Z = int(z_start*win_size)
                result = K.max(inputs[:,X:X+win_size,Y:Y+win_size,Z:Z+win_size,:],axis = (1,2,3))
                output.append(result)
      output = K.concatenate(output)
      return output

In [19]:
data1 = np.load(r'H:\Users\admin\Documents\NSU-PhD\Dissertation Proposal\CMB_Projeto\roi_01-10_16x16x10.npy', allow_pickle=True)
data2 = np.load(r'H:\Users\admin\Documents\NSU-PhD\Dissertation Proposal\CMB_Projeto\roi_11-20_16x16x10.npy', allow_pickle=True)

In [20]:
data = np.concatenate((data1,data2))
data.shape

(74, 16, 16, 10)

In [21]:
data_false = np.load(r'H:\Users\admin\Documents\NSU-PhD\Dissertation Proposal\CMB_Projeto\negatives_16x16x10.npy', allow_pickle=True)
data_false.shape

(160, 16, 16, 10)

In [22]:
fulldata = np.concatenate((data,data_false))
fulldata.shape

(234, 16, 16, 10)

In [23]:
ones = [1 for i in range(data.shape[0])]
zeros = [0 for i in range(data_false.shape[0])]
y = ones + zeros
y = np.array(y)
y = y.reshape(-1,1)
y.shape

(234, 1)

In [24]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder().fit(y)
onehotlabels = enc.transform(y).toarray()
onehotlabels.shape

(234, 2)

In [25]:
fulldata_res = fulldata.reshape(234,16,16,10,1)

In [26]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(fulldata_res, onehotlabels, test_size=0.3,shuffle= True)

In [27]:
ishape = (16,16,10,1)

In [28]:
import  os
import  tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, optimizers, models
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.models import Model
from tensorflow.python.keras import backend as K
#from keras_pyramid_pooling_module import PyramidPoolingModule

In [29]:
strides=(1, 1, 1)
padding = 'valid'

In [ ]:
model = tf.keras.Sequential()

#model.add(Input(shape=input_shape))
model.add(Conv3D(32, (7,7,5), strides=strides, padding=padding, input_shape=ishape))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(64, (5,5,3), strides=strides, padding=padding))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(MaxPooling3D(pool_size=(1,1,1)))

## Include SPP
model.add(SppnetLayer([1,1,1,64]))

model.add(Flatten())
model.add(Dense(500))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(2, activation='softmax'))
          
model.summary()

1.0
1.0
1.0
0.015625


In [ ]:
#### keras training
# compiling the optimizer, loss function, and  metrics 
model.compile(optimizer=tf.keras.optimizers.Adam(0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

batch_size = 100
epochs = 1000

check = ModelCheckpoint(filepath='./weights_seq2keras21_09.hdf5', verbose=0, save_best_only=True, monitor='val_loss')
early = EarlyStopping(monitor='val_loss',patience=3)

# train
history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs,validation_data=(X_test, y_test),
         verbose=1, callbacks=[ check, early])

In [ ]:
# evaluate on test set
scores = model.evaluate(X_test, y_test, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

In [ ]:
model.save(r'H:\Users\admin\Documents\NSU-PhD\Dissertation Proposal\CMB_Projeto\mymodel71accseq21_09.hdf5')

In [ ]:
model.save(r'H:\Users\admin\Documents\NSU-PhD\Dissertation Proposal\CMB_Projeto\mymodel78seq21_09')

In [ ]:
model2 = tf.keras.models.load_model(r'H:\Users\admin\Documents\NSU-PhD\Dissertation Proposal\CMB_Projeto\mymodel78seq21_09')

In [ ]:
# evaluate on test set
scores = model2.evaluate(X_test, y_test, batch_size, verbose=1)
print("Final test loss and accuracy :", scores)

In [ ]:
for i in history.history:
    print(i)

In [ ]:
import matplotlib.pyplot as plt
# Plot history: Categorical crossentropy & Accuracy
plt.plot(history.history['loss'], label='Categorical crossentropy (training data)')
#plt.plot(history.history['val_loss'], label='Categorical crossentropy (validation data)')
#plt.plot(history.history['accuracy'], label='Accuracy (training data)')
#plt.plot(history.history['val_acc'], label='Accuracy (validation data)')
plt.title('Model performance Conv3D example')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()

In [ ]:
import matplotlib.pyplot as plt
# Plot history: Categorical crossentropy & Accuracy
#plt.plot(history.history['loss'], label='Categorical crossentropy (training data)')
#plt.plot(history.history['val_loss'], label='Categorical crossentropy (validation data)')
plt.plot(history.history['accuracy'], label='Accuracy (training data)')
#plt.plot(history.history['val_acc'], label='Accuracy (validation data)')
plt.title('Model performance Conv3D example')
plt.ylabel('Loss value')
plt.xlabel('No. epoch')
plt.legend(loc="upper left")
plt.show()